Troubleshooting your databases by making sure you have privledges for your user in mariadb

In [ ]:
sudo mariadb -u root -p < ~/cats.sql # if you have cats.sql in your main home use the tilda or go through the ab
GRANT ALL PRIVILEGES ON catsrus.* TO 'annelise'@'localhost';

In [ ]:
pip install mysql-connector-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 30.6 MB/s eta 0:00:00


In [ ]:
import mysql.connector
from werkzeug.security import generate_password_hash

# Configuration for your MariaDB connection
config = {
    'user': 'username',  # or your MariaDB username
    'password': 'password',  # your MariaDB password
    'host': 'localhost',  # or your MariaDB host if not local
    'database': 'database',  # your database name
    'raise_on_warnings': True
}

# Users and their passwords
users = {
    'username': 'securepassword',
    'username': 'securepassword',
    'username': 'securepassword',
    'username': 'securepassword'
}

try:
    # Connect to the database
    db = mysql.connector.connect(**config)
    cursor = db.cursor()

    # Insert each user and their hashed password into the database
    for user, password in users.items():
        hashed_password = generate_password_hash(password)
        email = f"{user.lower()}@website.com"  # Example email format
        sql = f"UPDATE tablename SET password_hash = '{hashed_password}'WHERE name = {user} AND email = {email})"
        cursor.execute(sql)

    db.commit()  # Make sure data is committed to the database
    print("Users inserted successfully")

except mysql.connector.Error as err:
    print(f"Error: {err}")

finally:
    if (db.is_connected()):
        cursor.close()
        db.close()
        print("Database connection is closed")


In [ ]:
#checking to see if your hash is in the database

In [ ]:
import mysql.connector

# Assume these are set from your environment or securely stored
config = {
    'user': 'root',
    'password': 'your_password',
    'host': 'localhost',
    'database': 'databasename',
    'raise_on_warnings': True
}

def get_user_hash(email):
    try:
        db = mysql.connector.connect(**config)
        cursor = db.cursor()
        query = "SELECT password_hash FROM table WHERE email = %s"
        cursor.execute(query, (email,))
        result = cursor.fetchone()  # Get the first result
        return result[0] if result else None
    except mysql.connector.Error as err:
        print(f"Error: {err}")
        return None
    finally:
        if db.is_connected():
            cursor.close()
            db.close()

# Example usage
user_email = 'username@website.com'
user_hash = get_user_hash(user_email)
print("Retrieved Hash:", user_hash)

In [ ]:
from flask import Flask, request, render_template, redirect, url_for, flash
import mysql.connector
from werkzeug.security import check_password_hash
from markupsafe import Markup
import random
import mysql.connector
import configparser
from flask_httpauth import HTTPBasicAuth
from werkzeug.security import generate_password_hash, check_password_hash

#you will change the static folder path here to your own static folder path
app = Flask(__name__, static_folder='/home/static')


config = configparser.ConfigParser()
configFile = 'contacts.cfg'
config.read(configFile)
database = config['database']
db = database['db']
dbHost=database['host']
dbUser=database['user']
dbPass=database['pass']
cnx = mysql.connector.connect(user=dbUser, password = dbPass, host= dbHost, database= db)

@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        username = request.form['username']
        password = request.form['password']
        conn = mysql.connector.connect(**db_config)
        cursor = conn.cursor()
        cursor.execute("SELECT password_hash FROM contactsTable WHERE name = %s", (username,))
        user = cursor.fetchone()
        cursor.close()
        conn.close()
        if user and check_password_hash(user[0], password):
            # Login successful
            return redirect(url_for('dashboard'))  # Redirect to another page or dashboard
        else:
            flash('Invalid username or password')
            return redirect(url_for('login'))
    return render_template('login.html')

@app.route('/dashboard')
def dashboard():
    return 'Welcome to the Dashboard!'

if __name__ == '__main__':
    app.run(debug=True)